# Домашнее задание 2. Классификация изображений.

В этом задании потребуется обучить классификатор изображений. Будем работать сдатасетом, название которого раскрывать не будем. Можете посмотреть самостоятельно на картинки, которые в есть датасете. В нём 200 классов и около 5 тысяч картинок на каждый класс. Классы пронумерованы, как нетрудно догадаться, от 0 до 199. Скачать датасет можно вот [тут](https://yadi.sk/d/BNR41Vu3y0c7qA).

Структура датасета простая -- есть директории train/ и val/, в которых лежат обучающие и валидационные данные. В train/ и val/ лежат директориии, соответствующие классам изображений, в которых лежат, собственно, сами изображения.
 
__Задание__. Необходимо выполнить любое из двух заданий

1) Добейтесь accuracy **на валидации не менее 0.44**. В этом задании **запрещено** пользоваться предобученными моделями и ресайзом картинок. 

2) Добейтесь accuracy **на валидации не менее 0.84**. В этом задании делать ресайз и использовать претрейн можно. 

Напишите краткий отчёт о проделанных экспериментах. Что сработало и что не сработало? Почему вы решили, сделать так, а не иначе? Обязательно указывайте ссылки на чужой код, если вы его используете. Обязательно ссылайтесь на статьи / блогпосты / вопросы на stackoverflow / видосы от ютуберов-машинлернеров / курсы / подсказки от Дяди Васи и прочие дополнительные материалы, если вы их используете. 

Ваш код обязательно должен проходить все `assert`'ы ниже.

Необходимо написать функции `train_one_epoch`, `train` и `predict` по шаблонам ниже (во многом повторяют примеры с семинаров).Обратите особое внимание на функцию `predict`: она должна возвращать список лоссов по всем объектам даталоадера, список предсказанных классов для каждого объекта из даталоалера и список настоящих классов для каждого объекта в даталоадере (и именно в таком порядке).

__Использовать внешние данные для обучения строго запрещено в обоих заданиях. Также запрещено обучаться на валидационной выборке__.


__Критерии оценки__: Оценка вычисляется по простой формуле: `min(10, 10 * Ваша accuracy / 0.44)` для первого задания и `min(10, 10 * (Ваша accuracy - 0.5) / 0.34)` для второго. Оценка округляется до десятых по арифметическим правилам. Если вы выполнили оба задания, то берется максимум из двух оценок.

__Бонус__. Вы получаете 5 бонусных баллов если справляетесь с обоими заданиями на 10 баллов (итого 15 баллов). В противном случае выставляется максимальная из двух оценок и ваш бонус равен нулю.

__Советы и указания__:
 - Наверняка вам потребуется много гуглить о классификации и о том, как заставить её работать. Это нормально, все гуглят. Но не забывайте, что нужно быть готовым за скатанный код отвечать :)
 - Используйте аугментации. Для этого пользуйтесь модулем `torchvision.transforms` или библиотекой [albumentations](https://github.com/albumentations-team/albumentations)
 - Можно обучать с нуля или файнтюнить (в зависимости от задания) модели из `torchvision`.
 - Рекомендуем написать вам сначала класс-датасет (или воспользоваться классом `ImageFolder`), который возвращает картинки и соответствующие им классы, а затем функции для трейна по шаблонам ниже. Однако делать это мы не заставляем. Если вам так неудобно, то можете писать код в удобном стиле. Однако учтите, что чрезмерное изменение нижеперечисленных шаблонов увеличит количество вопросов к вашему коду и повысит вероятность вызова на защиту :)
 - Валидируйте. Трекайте ошибки как можно раньше, чтобы не тратить время впустую.
 - Чтобы быстро отладить код, пробуйте обучаться на маленькой части датасета (скажем, 5-10 картинок просто чтобы убедиться что код запускается). Когда вы поняли, что смогли всё отдебажить, переходите обучению по всему датасету
 - На каждый запуск делайте ровно одно изменение в модели/аугментации/оптимайзере, чтобы понять, что и как влияет на результат.
 - Фиксируйте random seed.
 - Начинайте с простых моделей и постепенно переходите к сложным. Обучение лёгких моделей экономит много времени.
 - Ставьте расписание на learning rate. Уменьшайте его, когда лосс на валидации перестаёт убывать.
 - Советуем использовать GPU. Если у вас его нет, используйте google colab. Если вам неудобно его использовать на постоянной основе, напишите и отладьте весь код локально на CPU, а затем запустите уже написанный ноутбук в колабе. Авторское решение задания достигает требуемой точности в колабе за 15 минут обучения.
 
Good luck & have fun! :)

In [1]:

import torch, torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms
from torch.autograd import Variable
import torchvision.models as models
import matplotlib.pyplot as plt
import os, copy, numpy as np
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from time import time


import sys

%matplotlib inline
# You may add any imports you need

### Подготовка данных

In [2]:
#!unzip /content/drive/MyDrive/dataset

In [3]:
data_transforms = { 'train': transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4802, 0.4481, 0.3975), (1, 1, 1)), ]),
                    'val'  : transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4802, 0.4481, 0.3975), (1, 1, 1)), ]) }

data_dir = '/content/dataset/dataset'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64, shuffle=True, num_workers=2)
              for x in ['train', 'val']}

train_dataset = dataloaders['train']
val_dataset = dataloaders['val']

### Вспомогательные функции, реализация модели

In [4]:
def train_one_epoch(model, train_dataloader, optimizer, criterion, return_losses=True, device="cuda:0"):
    model.to(device).train()
    total_loss = 0
    num_batches = 0
    all_losses = []
    total_predictions = np.array([])
    total_labels = np.array([])
    
    with tqdm(total=len(train_dataloader), file=sys.stdout) as prbar:
        for images, labels in train_dataloader:
            # Move Batch to GPU
            images = images.to(device)
            labels = labels.to(device)
            predicted = model(images)
            loss = criterion(predicted, labels)

           

            # Update weights
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            accuracy = (predicted.argmax(1) == labels).float().mean()
            prbar.set_description(f"Loss: {round(loss.item(), 4)} "
                                  f"Accuracy: {round(accuracy.item() * 100, 4)}")
            prbar.update(1)

            # Update descirption for tqdm
            accuracy = (predicted.argmax(1) == labels).float().mean()

            total_loss += loss.item()
            total_predictions = np.append(total_predictions, predicted.argmax(1).cpu().detach().numpy())
            total_labels = np.append(total_labels, labels.cpu().detach().numpy())
            num_batches += 1
            all_losses.append(loss.detach().item())
        
    metrics = {"loss": total_loss / num_batches}
    metrics.update({"accuracy": (total_predictions == total_labels).mean()})
    
    if return_losses:
        return metrics, all_losses
    else:
        return metrics



def validate(model, data_loader, criterion, device="cuda:0"):
    model.eval()
    # YOUR CODE
    # PREDICT FOR EVERY ELEMENT OF THE VAL DATALOADER AND RETURN CORRESPONDING LISTS
    total_loss = np.array([])
    num_batches = 0
    total_predictions = np.array([])
    total_labels = np.array([]) 
    m = []
    
    with tqdm(total=len(data_loader), file=sys.stdout) as prbar:
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)
            predicted = model(images)

            loss = criterion(predicted, labels)
            accuracy = (predicted.argmax(1) == labels).float().mean()
            
            prbar.set_description(f"Loss: {round(loss.item(), 4)} "
                                  f"Accuracy: {round(accuracy.item() * 100, 4)}")
            prbar.update(1)

            #print('loss', loss.item())
            total_loss = np.append(total_loss, loss.item())
            #print('total_loss', total_loss)
            total_predictions = np.append(total_predictions, predicted.argmax(1).cpu().detach().numpy())
            total_labels = np.append(total_labels, labels.cpu().detach().numpy())
            num_batches += 1
        
    metrics = {"losses": total_loss}
    metrics.update({"predicted_classes": total_predictions})
    metrics.update({"true_classes":  total_labels})
    m.append(total_loss)
    m.append(total_predictions)
    m.append(total_labels)
    return m

In [5]:
def train(model, train_dataloader, val_dataloader, criterion, optimizer, device="cuda:0", epochs=5, scheduler=None):
    model.to(device)
    all_train_losses = []
    epoch_train_losses = []
    epoch_eval_losses = []
    
    for epoch in range(epochs):
        # Train step
        print(f"Train Epoch: {epoch}")
        train_metrics, one_epoch_train_losses = train_one_epoch(model = model,
                                                                train_dataloader = train_dataloader,
                                                                optimizer=optimizer,
                                                                criterion=criterion,
                                                                return_losses=True,
                                                                device=device)
        
        # Save Train losses
        all_train_losses.extend(one_epoch_train_losses)
        epoch_train_losses.append(train_metrics["loss"])
        
        # Eval step
        print(f"Validation Epoch: {epoch}")
        with torch.no_grad():
            validation_metrics = validate(model=model, data_loader=val_dataloader, criterion=criterion, device="cuda:0")
            
        # Save eval losses
        epoch_eval_losses.append(validation_metrics[0])
        #print(f"Epoch: {epoch}, metrics: {validation_metrics}")
        
        #metrics = {"loss": total_loss / num_batches}
        accuracy = ({"accuracy": (validation_metrics[1] == validation_metrics[2]).mean()})

        print('Точность', accuracy)

### Обучение модели, запуски экспериментов

In [6]:
from torchvision.models import resnet18
model = models.resnet18(pretrained=False)

model.avgpool = nn.AdaptiveAvgPool2d(1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 200)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

model = torch.nn.DataParallel(model, device_ids=[0])

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [7]:
train(model, train_dataset, val_dataset, criterion, optimizer, device, 5, scheduler)

Train Epoch: 0
Loss: 3.7744 Accuracy: 12.5: 100%|██████████| 1563/1563 [02:47<00:00,  9.31it/s]
Validation Epoch: 0
Loss: 3.5076 Accuracy: 18.75: 100%|██████████| 157/157 [00:07<00:00, 21.27it/s]
Точность {'accuracy': 0.1658}
Train Epoch: 1
Loss: 2.9477 Accuracy: 31.25: 100%|██████████| 1563/1563 [02:47<00:00,  9.34it/s]
Validation Epoch: 1
Loss: 3.7412 Accuracy: 18.75: 100%|██████████| 157/157 [00:07<00:00, 21.84it/s]
Точность {'accuracy': 0.2351}
Train Epoch: 2
Loss: 3.1324 Accuracy: 25.0: 100%|██████████| 1563/1563 [02:46<00:00,  9.36it/s]
Validation Epoch: 2
Loss: 3.6485 Accuracy: 25.0: 100%|██████████| 157/157 [00:07<00:00, 21.75it/s]
Точность {'accuracy': 0.293}
Train Epoch: 3
Loss: 2.6217 Accuracy: 37.5: 100%|██████████| 1563/1563 [02:47<00:00,  9.36it/s]
Validation Epoch: 3
Loss: 2.1522 Accuracy: 37.5: 100%|██████████| 157/157 [00:07<00:00, 20.89it/s]
Точность {'accuracy': 0.3213}
Train Epoch: 4
Loss: 2.0806 Accuracy: 46.875: 100%|██████████| 1563/1563 [02:47<00:00,  9.32it/s]


Сделаю еще 3 эпохи

In [9]:
train(model, train_dataset, val_dataset, criterion, optimizer, device, 3, scheduler)

Train Epoch: 0
Loss: 2.0827 Accuracy: 53.125: 100%|██████████| 1563/1563 [02:48<00:00,  9.26it/s]
Validation Epoch: 0
Loss: 3.5157 Accuracy: 18.75: 100%|██████████| 157/157 [00:07<00:00, 20.62it/s]
Точность {'accuracy': 0.3511}
Train Epoch: 1
Loss: 1.2563 Accuracy: 68.75: 100%|██████████| 1563/1563 [02:49<00:00,  9.20it/s]
Validation Epoch: 1
Loss: 3.8886 Accuracy: 37.5: 100%|██████████| 157/157 [00:07<00:00, 20.86it/s]
Точность {'accuracy': 0.3392}
Train Epoch: 2
Loss: 0.6701 Accuracy: 78.125: 100%|██████████| 1563/1563 [02:49<00:00,  9.22it/s]
Validation Epoch: 2
Loss: 2.8344 Accuracy: 25.0: 100%|██████████| 157/157 [00:07<00:00, 20.76it/s]
Точность {'accuracy': 0.3333}


### Проверка полученной accuracy

In [10]:
with torch.no_grad():
  validation_metrics = validate(model=model, data_loader=val_dataset, criterion=criterion, device="cuda:0")

all_losses, predicted_labels, true_labels = validation_metrics[0], validation_metrics[1], validation_metrics[2]
assert len(predicted_labels) == len(true_labels)
accuracy = accuracy_score(predicted_labels, true_labels)
print("tests passed")

print('Точность:', accuracy)
print("Оценка за это задание составит {} баллов".format(min(10, 10 * (accuracy)) / 0.4))

Loss: 4.2835 Accuracy: 25.0: 100%|██████████| 157/157 [00:07<00:00, 20.98it/s]
tests passed
Точность: 0.3333
Оценка за это задание составит 8.3325 баллов


### Отчёт об экспериментах 


Мой подход - сначала оптимизировал предобученную модель до 0.76, потом на полученной архитектуре и без ресайза сделать pretrained = False.

Что помогло:

- смена последних слоев: добавил AdaptiveAvgPool2d(1) и поменял последний слой, чтобы resnet смог выдавать 200 классов. AdaptiveAvgPool2d показал наилучшие результаты

- Нормализация картинок дала значительный прирост

Что не помогло:

- Другие модели: VGG16, EfficientNet работали хуже, несмотря на то, что в статьях было указано обратное

- другие оптимизаторы: Adam и RMSprop давали результаты хуже, чем обычный SGD

- Аугментации не приводили к значительному приросту точности, оптимизации архитектуры более влиятельный подход

- Кроп не давал никаких результатов, хоть и нам всё подсказывали его юзать

